# Example: LFPy test suite

This example notebook submits a job to a chosen compute cluster (**JUSUF**), loads a **Singularity** container with a working **LFPy** installation and runs **LFPy**'s built in unit test suite (`LFPy.run_tests()`) before exiting. 

For more indepth info on using HPC backends via EBRAINS see https://wiki.ebrains.eu/bin/view/Collabs/using-supercomputers-from-the-collab/

In [ ]:
# use the pyunicore library
!pip install pyunicore --upgrade

In [ ]:
# import modules
import os
import pyunicore.client as unicore_client
import requests
import json
from time import sleep, time
from pprint import pprint

In [ ]:
unicore_client._HBP_REGISTRY_URL

In [ ]:
# Create connection to supercomputer (e.g., JUSUF)
tr = unicore_client.Transport(clb_oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)

In [ ]:
# Valid choices for supercomputers are one of the keys in:
r.site_urls

In [ ]:
# create client
supercomputer = 'JUSUF'
try:
    site_client = r.site(supercomputer)
except KeyError:
    site_client = unicore_client.Client(r.transport, 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core')

In [ ]:
# check connection to supercomputer
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url=site_client.site_url, headers=headers, verify=False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

In [ ]:
# check connection to supercomputer
# base = r.site_urls[supercomputer]
headers = {}
headers["Authorization"] = "Bearer " + clb_oauth.get_token()
headers['Accept'] = "application/json"
rs = requests.get(url='https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core', headers=headers, verify=False)
print("Status code %s " % rs.status_code)
print("Content-type %s " % rs.headers['Content-Type'])
reply = rs.json()
# print(json.dumps(reply, indent = 1))

## Prepare singularity container
See https://gitlab.version.fz-juelich.de/bvonstvieth_publications/container_userdoc_tmp for details. 

This step builds the singularity container. It is optional if the recipe has already been uploaded and built on the system. 

The procedure may be different on different HPC backends. 
The container can either way be built from the same recipe: https://raw.githubusercontent.com/LFPy/LFPydebian/main/mpich.Dockerfile

Turn the below Raw block into Code in order to run:

## Prepare main simulation job

This step combines in a single session the following:

- download LFPy container
- ask for resources (# nodes, # MPI processes, # seconds runtime)
- execute simulation

In [ ]:
# create dictionary with job specification and define list of input files from this Collab
# simulation_job = {"Job type": "interactive"}
simulation_job = {}
simulation_inputs = []

In [ ]:
# Resources
simulation_job['Resources'] = {
    "Queue": "batch",
    "CPUs": "1",
    "Runtime": "600",
}

In [ ]:
# commands run on login node before job execution
simulation_job["User precommand"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
sib download --recipe-name lfpymaster
"""
simulation_job["RunUserPrecommandOnLoginNode"] = "true"

In [ ]:
# - set some environment variables
# - run the python code using interpreter embedded in container
simulation_job["Executable"] = """module use $OTHERSTAGES
module --force purge
module load Stages/2020
module load GCC Singularity-Tools
unset DISPLAY  # matplotlib may look for a nonexistant display on compute node 
singularity exec lfpymaster.sif python3 -c "import LFPy; LFPy.run_tests()"
"""

In [ ]:
# commands run after job is done
simulation_job["User postcommand"] = "" 
simulation_job["RunUserPostcommandOnLoginNode"] = "true"

In [ ]:
simulation_job

In [ ]:
# create job
job = site_client.new_job(job_description=simulation_job, inputs=simulation_inputs)

In [ ]:
# wait while job is running
while job.is_running():
    sleep(10)

In [ ]:
job.working_dir.listdir().keys()

In [ ]:
# STDERR output (if any)
stderr = job.working_dir.stat('stderr')
pprint(stderr.raw().readlines())

In [ ]:
# STDOUT output
stdout = job.working_dir.stat('stdout')
pprint(stdout.raw().readlines())

In [ ]:
# kill job, clean up files on the remote
job.delete()